In [6]:
pip install paho-mqtt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import paho.mqtt.subscribe as subscribe
import csv
import json
import sys
import os

# Raw MQTT topics
topics = ['time', 'temp', 'air_quality', 'humidity']

# Friendly column labels
labels = ['time', 'temp', 'air quality', 'humidity']

n = 6  # number of cycles

csv_file = "mqtt_data_test.csv"
json_file = "mqtt_data_test.json"

# -----------------------
# SET UP CSV
# -----------------------
write_csv_header = not os.path.exists(csv_file)
csv_fh = open(csv_file, "a", newline="")
csv_writer = csv.writer(csv_fh)

if write_csv_header:
    csv_writer.writerow(["cycle"] + labels)

# -----------------------
# SET UP JSON
# -----------------------
json_fh = open(json_file, "a")

# -----------------------
# STORAGE FOR FINAL TABLE
# -----------------------
table_rows = []   # list of lists: [cycle, time, temp, aq, humidity]

# -----------------------
# MAIN LOOP
# -----------------------
for i in range(n):
    # Read MQTT messages
    m = subscribe.simple(
        topics,
        hostname="test.mosquitto.org",
        retained=False,
        msg_count=len(topics)
    )

    values = [msg.payload.decode("utf-8") for msg in m]

    # Store for final table
    table_rows.append([i] + values)

    # Write CSV
    csv_writer.writerow([i] + values)

    # Write JSON (per cycle)
    obj = {"cycle": i}
    for label, value in zip(labels, values):
        obj[label] = value

    json_fh.write(json.dumps(obj) + "\n")

# Close files
csv_fh.close()
json_fh.close()

# -----------------------
# PRINT FINAL SINGLE TABLE
# -----------------------
print("\n==================== FULL DATA TABLE ====================\n")

# Header
header = ["cycle"] + labels
print(" | ".join(f"{h:<15}" for h in header))
print("-" * 80)

# Rows
for row in table_rows:
    print(" | ".join(f"{str(col):<15}" for col in row))

print("\n==========================================================\n")